In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( )

In [ ]:
%matplotlib widget
import h5py, hdf5plugin
import numpy as np, matplotlib.pyplot as plt
import silx.io
import scipy.ndimage, fabio
import ImageD11.frelon_peaksearch
import ImageD11.blobcorrector
import ipywidgets
import ImageD11.transformer

In [ ]:
masterfile = "/data/id11/3dxrd/ihma233/id11/CeO2/CeO2_align/CeO2_align.h5"
scan = "39.1"
detector = "frelon3"
energy = 43.35
print(energy)

In [ ]:
darkscan = "34.1" # first 7 frames:
dark = silx.io.get_data( f"silx:{masterfile}::{darkscan}/measurement/{detector}" )[0:7].mean(axis=0, dtype=np.float32)

In [ ]:
frames = silx.io.get_data( f"silx:{masterfile}::{scan}/measurement/{detector}" )
frames.shape

In [ ]:
if detector.find('frelon')>=0:
    
    serial = silx.io.get_data( f"silx:{masterfile}::{darkscan}/instrument/{detector}/camera_settings/camera_serial" )
    print("Detector serial", serial )
    if serial == "21":
        flat = silx.io.get_data( f"fabio:/data/id11/3dxrd/inhouse/Frelon21/F21_flat_july18.edf" )
        e2dxfile = "/data/id11/3dxrd/inhouse/Frelon21/F21dxnew.edf"
        e2dyfile = "/data/id11/3dxrd/inhouse/Frelon21/F21dynew.edf"
        pixelsize = float(fabio.open(e2dxfile).header['PSize_1'].split()[0])*1e6 # microns
        print('Pixel size', pixelsize )
    if serial == "36": 
        flat = silx.io.get_data( f"fabio:/data/id11/3dxrd/inhouse/Frelon36/F36_Nov2023.edf" )
        flat = flat / flat[500:-500, 500:-500].mean()
        e2dxfile = "fabio:/data/id11/3dxrd/inhouse/Frelon36/f36dx.edf"
        e2dyfile = f"fabio:/data/id11/3dxrd/inhouse/Frelon36/f36dy.edf"
        pixelsize = float(fabio.open(e2dxfile).header['PSize_1'].split()[0])*1e6 # microns
    if serial == "29":
        flat = silx.io.get_data( f"fabio:/data/id11/3dxrd/inhouse/Frelon4M/c29_Agfilt_flat-dark.edf" )
        e2dxfile = "fabio:/data/id11/3dxrd/inhouse/Frelon4M/F4M_EO_dx.edf"
        e2dyfile = "fabio:/data/id11/3dxrd/inhouse/Frelon4M/F4M_EO_dy.edf"
        pixelsize = float(fabio.open(e2dxfile).header['PSize_1'].split()[0])*1e6 # microns
        
    print( "Average flat, should be close to 1 : ",flat[100:-100, 100:-100].mean() )
        
    cor = ( frames - dark[ None, :, :] ).mean( axis = 0, dtype = np.float32 ) / flat
    
else: # eiger
    
    cor = frames.mean( axis = 0, dtype = np.float32 )
    
    e2dxfile = "/data/id11/nanoscope/Eiger/e2dx_E-08-0144_20240205.edf"
    e2dyfile = "/data/id11/nanoscope/Eiger/e2dy_E-08-0144_20240205.edf"

In [ ]:
def guess_median_bg( img, size=91 ):
    m0 = scipy.ndimage.median_filter( cor, (size,1))
    m1 = scipy.ndimage.median_filter( cor, (1,size))
    bg = np.where(m0<m1,m0,m1)
    return bg

In [ ]:
lowcut = 200 # low cutoff for masking, check next plot
mask = ( cor < 4e9 ) & (cor > lowcut )

bg = guess_median_bg( cor )

signal = ((cor - bg )*mask).clip(0,None)

In [ ]:
f,(a,b) = plt.subplots( 2, 2, figsize=(8, 6), constrained_layout=True )
f.colorbar( a[1].imshow( cor * mask ) )
f.colorbar( b[1].imshow( signal ) )
a[0].plot( cor[ 1024, : ] )
a[0].plot( cor[ :, 1024 ] )
a[0].set(xlabel='pixel', ylabel='intensity')
b[0].plot( signal[ 1024, : ] )
b[0].plot( signal[ :, 1024 ] )
b[0].set(xlabel='pixel', ylabel='intensity', ylim=(0, None) );

In [ ]:
# Old way, using command line:
# !powderimagetopeaks.py CeO2_for_S5_blue_rot_clean.edf CeO2_for_S5_blue_rot_clean0000.edf CeO2_for_S5_blue_rot_clean0001.edf 1024 1024
def powderimagetopeaks( img, ci = 1024, cj = 1024 ):
    # cuts into 1 degree bins
    si, sj = img.shape
    i, j = np.mgrid[0:si,0:sj]
    phi = np.arctan2( i - ci, j - cj ) * 361 / np.pi
    return (phi%2).astype(int) == 0

In [ ]:
# old way:
# !peaksearch.py -n CeO2_for_S5_blue_rot_clean -f 0 -l 1 -p N -s /data/id11/3dxrd/inhouse/Frelon36/frelon36.spline -t 5000 -t 10000 -t 20000 -t 40000 -o CeO2_for_S5_blue_rot_clean.spt 

In [ ]:
worker = ImageD11.frelon_peaksearch.worker(None, None)
pkmsk = powderimagetopeaks( signal )
pks = np.concatenate( (
     worker.peaksearch( signal * pkmsk ),     # even degrees
     worker.peaksearch( signal * (1-pkmsk)), # odd degrees
    ) , axis = 0 )
pks.shape

In [ ]:
from ImageD11.cImageD11 import s_1, s_I, s_sI, s_fI
colfile = ImageD11.columnfile.colfile_from_dict( {
    "s_raw" : pks[ :, s_sI ] / pks[ :, s_I ],
    "f_raw" : pks[ :, s_fI ] / pks[ :, s_I ],
    "Number_of_pixels" : pks[ :, s_1 ], 
    "sum_intensity" : pks[ :, s_I ],
    "omega" : np.zeros( len(pks), float ),
} )

In [ ]:
f, a = plt.subplots(1,2, figsize=(8,5), constrained_layout=True)
a[0].plot( colfile.f_raw, colfile.s_raw, ".", ms = 1)
a[0].set( ylabel="detector slow direction", xlabel="detector fast direction", aspect='equal')
a[1].plot( colfile.Number_of_pixels, colfile.sum_intensity, ".", ms =1 )
a[1].set( xlabel="Number of pixels", ylabel="sum intensity", yscale='log', xscale='log');

In [ ]:
# old way
# !spatialfix.py frelon36_spline_dx.edf frelon36_spline_dy.edf CeO2_for_S5_blue_rot_clean_t5000.flt CeO2_for_S5_blue_rot_clean_spline_t5000.flt

In [ ]:
colfile = ImageD11.blobcorrector.eiger_spatial( e2dxfile, e2dyfile )(colfile)

In [ ]:
# input data from peaksearch and starting parameter file:
trans=ImageD11.transformer.transformer()
trans.colfile = colfile
trans.setxyomcols("sc", "fc", "omega")

In [ ]:
distance = 210 * 1e3 # micron
wavelength = 12.3984 / energy

In [ ]:
wavelength, energy

In [ ]:
trans.parameterobj.set( 'y_size', pixelsize )
trans.parameterobj.set( 'z_size', pixelsize )
trans.parameterobj.set( 'distance', distance )
trans.parameterobj.set( 'wavelength', wavelength )

In [ ]:
# unit cell
a = 5.4115260
[ trans.parameterobj.set( f'cell__{x}', a ) for x in 'abc']
[ trans.parameterobj.set( f'cell_{x}', 90 ) for x in ('alpha','beta','gamma')]
trans.parameterobj.set('cell_lattice_[P,A,B,C,I,F,R]', "F" )

In [ ]:
def parCallBack( arg ):
    trans.parameterobj.parameters.update( { arg['owner'].description : arg['new'] } )
    drawPlot()
    
def fixVaryCallBack( arg ):
    name = arg['owner'].description.split(" ")[1]
    vars = trans.getvars()
    if arg.new and not (name in vars):
        vars.append( name )
    if name in vars and not arg.new:
        vars.remove(name)
    trans.parameterobj.set_varylist(vars)
    
def fitCallBack(arg):
    """ fit call back - runs fit """
    lo, hi = ax1.get_xlim()
    trans.fit( lo, hi )
    for i, pname in enumerate(vars):
        layout[i,0].value = trans.parameterobj.get(pname)
    drawPlot()
    
def drawPlot():
    tth, eta = trans.compute_tth_eta()
    pt1.set_data( tth, eta )
    fig.canvas.update

# Things to be edited in the UI
vars = "y_center z_center distance tilt_y tilt_z".split()
steps = (1,         1,       100,     0.01, 0.01, 0)
nv = len(vars)

# Draw the widgets:
layout = ipywidgets.GridspecLayout(nv+1,2)
for i,( pname, pstep ) in enumerate( zip( vars, steps ) ) :
    layout[i,0] = ipywidgets.FloatText( description=pname, 
        value = trans.parameterobj.parameters.get(pname),
        step=pstep)
    layout[i,0].observe( parCallBack , names='value' )
    layout[i,1] = ipywidgets.ToggleButton( description="Vary "+pname, 
        value = pname in trans.getvars() )
    layout[i,1].observe( fixVaryCallBack, names='value' )
    
layout[nv,0] = ipywidgets.FloatText( description='fit_tolerance', 
        value = trans.parameterobj.parameters.get("fit_tolerance"), step=0,)
layout[nv,0].observe( parCallBack , names='value' )

layout[nv,1] = ipywidgets.Button(description="Run Fit (blocks)")
layout[nv,1].on_click( fitCallBack )


# Draw plot
fig1 = plt.figure(figsize=(9,6))
ax1 = fig1.add_subplot()
tth, eta = trans.compute_tth_eta()
trans.addcellpeaks()
pt1, = ax1.plot( tth, eta, ",")
ax1.set(xlabel="tth", ylabel="eta")
ax1.plot( trans.theorytth, [0,]*len(trans.theorytth), "r|", ms=360, alpha=0.2 )
# Add controls
display(layout)

In [ ]:
import pprint
pprint.pprint( trans.parameterobj.parameters )

In [ ]:
trans.parameterobj.saveparameters('CeO2_fitted.par')

In [ ]:
12.3985 / trans.parameterobj.get('wavelength')